# AES FPGA - testing

## Overlay
Requires .bit and .tcl files.

Syntax:
```python
from pynq.overlays.base import BaseOverlay
overlay = BaseOverlay("base.bit")
```

In [1]:
from pynq import Overlay
from pynq import MMIO

In [2]:
# Dowload bit file to PYNQ
bit_file_name = "design_1_wrapper.bit"
base = Overlay(bit_file_name)

base.download()
base.is_loaded()

/usr/local/lib/python3.6/dist-packages/pynq/overlay.py:299: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


True

## `mmap` interface

In [3]:
# mmap config
base_address = 0x43C00000
ctrl_size = 2**10     # 1kb control register memory
dmem_size = 32*2**10  # 32kb (15-bits)
imem_size = 32*2**10  # 32kb (15-bits)
pmem_size = 4*2**10   # 4kb paramter memory

region_size = max([ctrl_size, dmem_size, imem_size, pmem_size])

ctrl_mmap = MMIO(base_address, ctrl_size)
imem_mmap = MMIO(base_address + region_size, imem_size)
dmem_mmap = MMIO(base_address + (region_size*2), dmem_size)
pmem_mmap = MMIO(base_address + (region_size*3), pmem_size)

# print control memory segments
print ("DEVICECLASS : " + str(ctrl_mmap.read(0x300)))
print ("DEVICE_ID : " + str(ctrl_mmap.read(0x304)))
print ("INTERFACE_TYPE : " + str(ctrl_mmap.read(0x308)))
print ("CORE_COUNT : " + str(ctrl_mmap.read(0x30C)))
print ("CTRL_SIZE : " + str(ctrl_mmap.read(0x310)))
print ("DMEM_SIZE : " + str(ctrl_mmap.read(0x314)))
print ("IMEM_SIZE : " + str(ctrl_mmap.read(0x318)))
print ("PMEM_SIZE : " + str(ctrl_mmap.read(0x31C)))

DEVICECLASS : 1908
DEVICE_ID : 305419896
INTERFACE_TYPE : 2
CORE_COUNT : 1
CTRL_SIZE : 1024
DMEM_SIZE : 32768
IMEM_SIZE : 262144
PMEM_SIZE : 1


In [162]:
# mmap config based on the TTA core's value
base_address = 0x43C00000
ctrl_size = 1024     # 1kb control register memory
dmem_size = 32768    # 32kb (15-bits)
imem_size = 262144   # Not sure why.
pmem_size = 1        # Not sure about the reason

region_size = max([ctrl_size, dmem_size, imem_size, pmem_size])

ctrl_mmap = MMIO(base_address, ctrl_size)
imem_mmap = MMIO(base_address + region_size, imem_size)
dmem_mmap = MMIO(base_address + (region_size*2), dmem_size)
pmem_mmap = MMIO(base_address + (region_size*3), pmem_size)

# print control memory segments
print ("DEVICECLASS : " + str(ctrl_mmap.read(0x300)))
print ("DEVICE_ID : " + str(ctrl_mmap.read(0x304)))
print ("INTERFACE_TYPE : " + str(ctrl_mmap.read(0x308)))
print ("CORE_COUNT : " + str(ctrl_mmap.read(0x30C)))
print ("CTRL_SIZE : " + str(ctrl_mmap.read(0x310)))
print ("DMEM_SIZE : " + str(ctrl_mmap.read(0x314)))
print ("IMEM_SIZE : " + str(ctrl_mmap.read(0x318)))
print ("PMEM_SIZE : " + str(ctrl_mmap.read(0x31C)))
print ("DEBUG_FEATURE : " + str(ctrl_mmap.read(0x320)))
print ("BREAKPOINT_COUNT : " + str(ctrl_mmap.read(0x320)))

DEVICECLASS : 1908
DEVICE_ID : 305419896
INTERFACE_TYPE : 2
CORE_COUNT : 1
CTRL_SIZE : 1024
DMEM_SIZE : 32768
IMEM_SIZE : 262144
PMEM_SIZE : 1
DEBUG_FEATURE : 1
BREAKPOINT_COUNT : 1


## Read data to memory

In [5]:
# Write to instruction memory
imem_img = "aes.img"
i=0
with open (imem_img, mode='r') as file:
    for line in file:
        line = line.strip().rjust(64,'0')
        for j in [1, 0]:
            i_word = int(line[(32*j):(32*j+32)], 2)
            imem_mmap.write(i, i_word)
            i += 4

In [6]:
# Write to data memory
dmem_img = "aes_data.img"
i=0
with open (dmem_img, mode='r') as file:
    for line in file:
        line = line.strip().rjust(32,'0')
        i_word = int(line, 2)
        dmem_mmap.write(i, i_word)
        i += 4

In [7]:
def printTTAStatus(addr):
    ctrl_mmap = addr
    print ("STATUS        : " + str(ctrl_mmap.read(0x0)))
    print ("CYCLECNT      : " + str(ctrl_mmap.read(0x4)))
    print ("CYCLECNT_HIGH : " + str(ctrl_mmap.read(0x8)))
    print ("LOCKCNT       : " + str(ctrl_mmap.read(0xC)))
    print ("LOCKCNT_HIGH  : " + str(ctrl_mmap.read(0x10)))

In [8]:
printTTAStatus(ctrl_mmap)

STATUS        : 0
CYCLECNT      : 0
CYCLECNT_HIGH : 0
LOCKCNT       : 0
LOCKCNT_HIGH  : 0


In [9]:
# Trigger exeuction
ctrl_mmap.write(0x200, 0x2)

In [44]:
print(hex(dmem_mmap.read(8)))

0x62488e82


In [165]:
# Trigger exeuction
ctrl_mmap.write(0x200, 0x1)
# Trigger exeuction
ctrl_mmap.write(0x200, 0x2)
printTTAStatus(ctrl_mmap)

STATUS        : 0
CYCLECNT      : 17
CYCLECNT_HIGH : 97791
LOCKCNT       : 0
LOCKCNT_HIGH  : 2


In [166]:
print ("CUSTOM        : " + str(ctrl_mmap.read(0x0)))
print ("CUSTOM        : " + str(ctrl_mmap.read(0x4)))
print ("CUSTOM        : " + str(ctrl_mmap.read(0x8)))
print ("CUSTOM        : " + str(ctrl_mmap.read(0xc)))
print ("CUSTOM        : " + str(ctrl_mmap.read(0x10)))
print ("CUSTOM        : " + str(ctrl_mmap.read(0x14)))
print ("")
print ("CUSTOM2        : " + str(ctrl_mmap.read(0x200)))
print ("CUSTOM2        : " + str(ctrl_mmap.read(0x204)))
print ("CUSTOM2        : " + str(ctrl_mmap.read(0x208)))
print ("CUSTOM2        : " + str(ctrl_mmap.read(0x20c)))
print ("CUSTOM2        : " + str(ctrl_mmap.read(0x210)))

CUSTOM        : 0
CUSTOM        : 16
CUSTOM        : 94121475
CUSTOM        : 0
CUSTOM        : 2
CUSTOM        : 0

CUSTOM2        : 0
CUSTOM2        : 0
CUSTOM2        : 0
CUSTOM2        : 0
CUSTOM2        : 0


In [338]:
# Enable break-point 2 at address 17
ctrl_mmap.write(0x208,8)
ctrl_mmap.write(0x210,17)

# reset the counters
ctrl_mmap.write(0x200, 0x1)

# start the execution
ctrl_mmap.write(0x200, 0x2)

#### Wait for some time
# Read the counters you require
# Write instruction/data memory with new program
# Set break-point for new program
ctrl_mmap.write(0x210,25)

# reset the counters
ctrl_mmap.write(0x200, 0x1)

# start the execution
ctrl_mmap.write(0x200, 0x2)

In [339]:
print ("CUSTOM        : " + str(ctrl_mmap.read(0x0)))
print ("CUSTOM        : " + str(ctrl_mmap.read(0x4)))
print ("CUSTOM        : " + str(ctrl_mmap.read(0x8)))
print ("CUSTOM        : " + str(ctrl_mmap.read(0xc)))
print ("CUSTOM        : " + str(ctrl_mmap.read(0x10)))
print ("CUSTOM        : " + str(ctrl_mmap.read(0x14)))
print ("")
print ("CUSTOM2        : " + str(ctrl_mmap.read(0x200)))
print ("CUSTOM2        : " + str(ctrl_mmap.read(0x204)))
print ("CUSTOM2        : " + str(ctrl_mmap.read(0x208)))
print ("CUSTOM2        : " + str(ctrl_mmap.read(0x20c)))
print ("CUSTOM2        : " + str(ctrl_mmap.read(0x210)))

CUSTOM        : 2
CUSTOM        : 17
CUSTOM        : 24
CUSTOM        : 0
CUSTOM        : 47
CUSTOM        : 0

CUSTOM2        : 0
CUSTOM2        : 0
CUSTOM2        : 8
CUSTOM2        : 17
CUSTOM2        : 17


In [292]:
print ("CUSTOM        : " + str(ctrl_mmap.read(0x0)))
print ("CUSTOM        : " + str(ctrl_mmap.read(0x4)))
print ("CUSTOM        : " + str(ctrl_mmap.read(0x8)))
print ("CUSTOM        : " + str(ctrl_mmap.read(0xc)))
print ("CUSTOM        : " + str(ctrl_mmap.read(0x10)))
print ("CUSTOM        : " + str(ctrl_mmap.read(0x14)))
print ("")
print ("CUSTOM2        : " + str(ctrl_mmap.read(0x200)))
print ("CUSTOM2        : " + str(ctrl_mmap.read(0x204)))
print ("CUSTOM2        : " + str(ctrl_mmap.read(0x208)))
print ("CUSTOM2        : " + str(ctrl_mmap.read(0x20c)))
print ("CUSTOM2        : " + str(ctrl_mmap.read(0x210)))

CUSTOM        : 0
CUSTOM        : 15
CUSTOM        : 2372663975
CUSTOM        : 0
CUSTOM        : 49
CUSTOM        : 0

CUSTOM2        : 0
CUSTOM2        : 0
CUSTOM2        : 0
CUSTOM2        : 17
CUSTOM2        : 17
